In [1]:
from typing import List, Dict, Any, TypedDict
from pm_market_getter import get_markets_for_date_range
from langchain.llms import OpenAI
from langchain.tools import Tool
from langchain.agents import AgentExecutor, OpenAIFunctionsAgent
from langchain.prompts import ChatPromptTemplate
from langchain.pydantic_v1 import BaseModel, Field
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolInvocation
from operator import add
from typing import Annotated
from datetime import datetime, timedelta
from langgraph.prebuilt import ToolExecutor
import requests
import json
import re
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
import operator
from typing import  Annotated
from langgraph.constants import Send
import threading

/Users/ethangoldberg/Desktop/CS-Proj/PolyNewsDaily/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
class Counter:
    def __init__(self):
        self._lock = threading.Lock()
        self._value = 0
    
    def get_next(self):
        with self._lock:
            self._value += 1
            return self._value

# Global counter instance
article_counter = Counter() 

class NewsArticles(BaseModel):
    title: str
    description: str
    source_name: str
class ArticleOutput(BaseModel):
    id: int = Field(default_factory=article_counter.get_next)
    headline: str
    subheader: str
    blurb: str = Field(default=None)
    score: int = Field(default=None)
    ticker: str = Field(default=None)
    links: List[str] = Field(default=None)

class MarketInfo(BaseModel):
    interest_score: float = Field(default=None)
    title: str = Field(default=None)
    ticker: str = Field(default=None)
    description: str = Field(default=None)
    end_date: str = Field(default=None)
    volume: float = Field(default=None)
    featured: float = Field(default=None)
    volume24hr: float = Field(default=None)
    commentCount: float = Field(default=None)
    options: List[Dict[str, Any]] = Field(default=None)
    tags: List[Dict[str, Any]] = Field(default=None)
    tavily_context: Annotated[list, operator.add] = Field(default=None)
    links: Annotated[list, operator.add] = Field(default=None)
    articles: List[NewsArticles] = Field(default_factory=list, description="Collected news articles")
    newsletter_content: List[ArticleOutput] = Field(default=None)
    skipped_newsletter_content: List[ArticleOutput] = Field(default=None)

class OverallGraph(BaseModel):
    markets_list: List[MarketInfo]
    newsletter_content: Annotated[List[ArticleOutput], operator.add] = Field(default=None)
    skipped_newsletter_content: Annotated[List[ArticleOutput], operator.add] = Field(default=None)
    article_groups: Dict[str, List[int]] = Field(
        default_factory=dict,
        description="Dictionary of lists, where each list contains ids of a group of articles. The string is the title of the group"
    )
    done: bool = Field(default = False)

class ExperimentState(TypedDict):
    market: dict  # The current market being processed
    article: dict  # The article being generated/refined
    previous_articles: List[dict]  # List of previously processed articles
    keep_article: bool

class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Search query for retrieval.")

In [3]:
from dotenv import load_dotenv
import os
import sys
from pathlib import Path

# Use the specific path to the .env file
env_path = Path('/Users/ethangoldberg/Desktop/CS-Proj/PolyNewsDaily/PolyNewsDaily/.env')

# Print current working directory for debugging
current_dir = Path.cwd()
print(f"Current working directory: {current_dir}")

# Load the .env file from the specific location
if env_path.exists():
    print(f"Found .env file at: {env_path}")
    load_dotenv(env_path)
    env_loaded = True
else:
    print(f"Could not find .env file at: {env_path}")
    env_loaded = False

# Get environment variables
news_api_key = os.getenv('NEWS_API_KEY')
openai_api = os.getenv('OPENAI_API_KEY')
tavily_api_key = os.getenv('TAVILY_API_KEY')
smtp_pass = os.getenv('SMTP_PASS')

# Print values to debug
print(f"SMTP_PASS: {'Found' if smtp_pass else 'Not found'}")
print(f"NEWS_API_KEY: {'Found' if news_api_key else 'Not found'}")
print(f"OPENAI_API_KEY: {'Found' if openai_api else 'Not found'}")
print(f"TAVILY_API_KEY: {'Found' if tavily_api_key else 'Not found'}")


Current working directory: /Users/ethangoldberg/Desktop/CS-Proj/PolyNewsDaily/PolyNewsDaily/agent
Found .env file at: /Users/ethangoldberg/Desktop/CS-Proj/PolyNewsDaily/PolyNewsDaily/.env
SMTP_PASS: Found
NEWS_API_KEY: Not found
OPENAI_API_KEY: Found
TAVILY_API_KEY: Found


In [5]:
print(tavily_api_key)


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


In [6]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
def generate_news_query(title, description) -> str:
    llm = ChatOpenAI(model="gpt-5", api_key=openai_api, temperature=1)  # gpt-5 requires default temp=1
    
    prompt = f"""
    You are an expert in information retrieval and natural language processing. Your task is to analyze the title and description of a prediction market and generate optimal keywords for a news API search. These keywords should maximize the retrieval of relevant and informative news articles about the market's subject.
bn
    Given:
    - Title: {title}
    - Description: {description}

    Instructions:
    1. Carefully read and understand the title and description.
    2. Identify the core subject matter, key entities, and central concepts.
    3. Consider potential synonyms, related terms, and broader categories that might yield relevant results.
    4. Avoid overly generic terms that might lead to irrelevant results.
    5. Include any specific dates, events, or proper nouns mentioned that are crucial to the market.
    6. Consider the timeframe of the prediction, if specified, and how it might affect keyword relevance.
    7. Aim for a balance between specificity (to ensure relevance) and breadth (to capture a range of potentially useful articles).
    8. Generate 1-3 keyword phrases, separated by commas if more than one.
    9. Each keyword should always be ONE word. There are exceptions for when two or three words are closely linked like "Donald Trump" is two words but would be allowed.

    Output your result in the following format. DO NOT say any other words or give any other explanation.
    Keywords: [Your generated keywords]
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    # Extract keywords from the response
    keywords_line = [line for line in response.content.split('\n') if line.startswith('Keywords:')][0]
    keywords = keywords_line.split(':', 1)[1].strip()
    keywords_array = [item.strip() for item in keywords.split(',')]
    return keywords_array

In [20]:
os.environ["TAVILY_API_KEY"] = tavily_api_key
tavily_getter = TavilySearchResults(max_results=10)

In [23]:
def tavily_search(state: MarketInfo):
    llm = ChatOpenAI(model="gpt-5", api_key=openai_api, temperature=1)
    search_instructions = """You will be given the title, description, and end_date for a polymarket prediction market. 

Your goal is to generate a well-structured query for use in retrieval and / or web-search related to the market to provide context to the market. For example, there may be a market on whether Taylor Swift gets married in 2024. A good search query might be "Taylor Swift Pregnancy Rumors 2024".
        
First, analyze the market.

Pay particular attention to the title, description, and dates for the market.

Convert this final question into a well-structured web search query that will give important context to understand it.

Title : {title}          

Description : {description}

End Date : {end_date}
                                        """
    structured_llm = llm.with_structured_output(SearchQuery)
    system_message = search_instructions.format(title=state.title, description=state.description, end_date = state.end_date)
    search_query = structured_llm.invoke([SystemMessage(content=system_message)])

    search_docs = tavily_getter.invoke(search_query.search_query)

    if isinstance(search_docs, str):
        print("Error: Expected a list of documents but got a string.")
        print("Response content:", search_docs)
        return {"tavily_context": [], "links": []} 

    formatted_search_docs = "\n\n---\n\n".join(
        [
            f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
            for doc in search_docs
        ]
    )
    links = [doc["url"] for doc in search_docs]
    print(links)
    return {"tavily_context": [formatted_search_docs],
            "links": links} 


In [8]:
def call_news_api(state: MarketInfo) -> MarketInfo:
    keywords = generate_news_query(state.title, state.description)
    print(keywords)
    today = datetime.now().date()
    from_date = (today - timedelta(days=3)).isoformat()
    to_date = today.isoformat()
    url = "https://newsapi.org/v2/everything"
    for keyword in keywords:
        params = {
            "from": from_date,
            "to": to_date,
            "apiKey": news_api_key,
            "country" : "us",
            "q" : keyword,
            "pageSize" : 2
        }
        response = requests.get(url, params=params)
        data = response.json()
        articles = data.get("articles", [])
        for article in articles:
            article_info =  NewsArticles(
            title=article['title'],
            description=article['description'],
            source_name=article['source']['name']
        )
            state.articles.append(article_info)

    return state    


In [9]:
class market_key_info(BaseModel):
    what_is_the_market_asking_or_answering: str = Field(defualt=None, description= "this should be a synposis of what this market is asking or answering")
    important_criteria: List[str] = Field(default_factory=List, description="a list of strings that describe important critera for a market and how it is resolved. includes timeline")
    completed: bool = Field(default=None, description="boolean indicating whether or not a market is essentially resolved. If the probability of all the options adds up to 0 or if the probability of any of the options is about 100, it is essentially guaranteed to already have happened or will happen")
    important_insights_context_or_reasoning : str = Field(default=None, description= "summary of important context or insights that could be related to this market. if there is important context from the sources from the internet regarding a probability or a change")

In [10]:
class article_score(BaseModel):
    score: int = Field(default=5, descrption="score for the article 0-10")

In [11]:
llm = ChatOpenAI(model="gpt-5", api_key=openai_api, temperature=1, parallel_tool_calls=False)

/Users/ethangoldberg/Desktop/CS-Proj/PolyNewsDaily/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3639: UserWarning: WARNING! parallel_tool_calls is not default parameter.
                parallel_tool_calls was transferred to model_kwargs.
                Please confirm that parallel_tool_calls is what you intended.
  if await self.run_code(code, result, async_=asy):


In [12]:
def generate_newsletter_content(market: MarketInfo) -> OverallGraph:
    market_dict = market.dict(exclude={'articles', 'final_answer', 'featured','commentCount', 'interest_score', 'newsletter_content', 'tavily_context'})
    context = market.tavily_context
    print(context)
    llm = ChatOpenAI(model="gpt-4.1", api_key=openai_api, temperature=1, parallel_tool_calls=False)
    intermediate_model = llm.with_structured_output(market_key_info)
    prompt = f"""
    It's great to finally meet you. You are my expert research analyst for my VC fund, specializing in reading and contextualizing prediciton markets. Be objective and precise. Be formal and exact. Your job is to take a structured api output from a prediction market and results from the internet for context on that market and: draw out the specific insights to later be used to create a news article. You should write in your output eveyrthing I need to draw from the market that is interesting.
    Here is the model you will be outputting:
    what_is_the_market_asking_or_answering: str = Field(defualt=None, description= "this should be a synposis of what this market is asking or answering")
    important_criteria: List[str] = Field(default_factory=List, description="a list of strings that describe important critera for a market and how it is resolved. includes timeline")
    completed: bool = Field(default=None, description="boolean indicating whether or not a market is essentially resolved. If the probability of all the options adds up to 0 or if the probability of any of the options is about 100, it is essentially guaranteed to already have happened or will happen")
    important_insights_context_or_reasoning : str = Field(default=None, description= "summary of important context or insights that could be related to this market. if there is important context from the sources from the internet regarding a probability or a change")

    The market_dict is a dictionary containing the following key information:
    - title: The title of the Polymarket prediction market
    - description: A description of the market
    - end_date: The resolution date for the market (pay close attention to this)
    - tags: Relevant tags for the market
    - options: A list of dictionaries, each containing:
      - 'outcome': The possible outcome
      - 'probability': The likelihood of the outcome (in decimal form, e.g., 0.75 = 75%)
      - 'oneDayPriceChange': How much the probability has changed in the last day

    The context variable is a list of sources and information from the internet.

    Here is the market to take to produce these insights:
    {market_dict}

    And here are the articles from the internet:

    {context}


    Remember to format your insights as described and be sure not to miss anything important. Be as compelling and interesting as possible.

    """
    response = intermediate_model.invoke([HumanMessage(content=prompt)])
    
    response_dict = response.dict()

    print("this is the intermediate reseponse: ", response_dict)
    
      # You can use a different model if preferred
    
    # Create a dict of the market model, omitting the articles
    
    prompt = f"""
You are a savvy AI newsletter writer specializing in prediction markets and current events. Your task is to generate compelling newsletter content based on Polymarket prediction markets and potentially related search context. Your writing should be engaging, insightful, and highlight the fascinating world of prediction markets, using context where applicable. Your audience is  intelligent and have signed up for your newsletter because they believe that prediction markets unlock information about the world and about the news.

The market_dict is a dictionary containing the following key information:
- title: The title of the Polymarket prediction market
- description: A description of the market
- end_date: The resolution date for the market (pay close attention to this)
- tags: Relevant tags for the market
- options: A list of dictionaries, each containing:
  - 'outcome': The possible outcome
  - 'probability': The likelihood of the outcome (in decimal form, e.g., 0.75 = 75%)
  - 'oneDayPriceChange': How much the probability has changed in the last day


You will get the following curated info about the market:
    what_is_the_market_asking_or_answering: str = Field(defualt=None, description= "this should be a synposis of what this market is asking or answering")
    important_criteria: List[str] = Field(default_factory=List, description="a list of strings that describe important critera for a market and how it is resolved. includes timeline")
    completed: bool = Field(default=None, description="boolean indicating whether or not a market is essentially resolved. If the probability of all the options adds up to 0 or if the probability of any of the options is about 100, it is essentially guaranteed to already have happened or will happen")
    important_insights_context_or_reasoning : str = Field(default=None, description= "summary of important context or insights that could be related to this market. if there is important context from the sources from the internet regarding a probability or a change")


Lastly, you will get information from the internet. This should be used as supplemental information and context as to the cause of certain odds, and when used in the article, should be cited.

Note: The interesting part of the market and how it relates to narratives on the internet might not need all of the provided information. Use your judgment to determine the applicability of information.

Instructions:
1. Carefully analyze the Polymarket prediction market information, paying special attention to the end_date and time frame criteria specified in the title or description.
2. Review the related context provided by searching the web, identifying any relevant information to the market. If no links are relevant, focus solely on the market data.
3. Identify the most interesting or surprising aspects of the market probabilities, potentially contrasting them with relevant news reporting if applicable.
4. Create a catchy headline that captures attention by highlighting these interesting aspects, ensuring it accurately reflects the time frame of the market.
5. Write a subheader that provides additional context or intrigue.
6. Compose a brief, engaging blurb that expands on the headline and subheader, incorporating relevant information from the prediction market and, if applicable, the web.

These are notes from an expert newsletter author on writing headlines that you should ABSOLUTELY use to make your headlines.
Overall:
3. Be active in your sentences not passive.
4. Do not use slang.
5. Use present tense and do not use articles (a, an, the)
6. Be professional, yet quippy and smart. Write like Packy Mccormick in Not Boring.

HEADLINE RULES:
- Must be news, not description (Example: "Biden wins vote" NOT "Vote held on bill")
- Write in positive form. (Example:
Dont write: Jonas didnt win the election.
Instead write: Jonas lost the election)
- Maximum 8 words
- Include most dramatic number/stat if available
- Use em dash (—) to add quick context
- Never use questions
- Start with the outcome/result
- No articles (a, an, the) at start

SUBHEADER RULES:
- Must add NEW information not in headline
- Never repeat headline information
- One line maximum
- Connect to headline's impact
- Include market implications or broader context
- Must explain "why it matters"

BLURB RULES:
- Second sentence: Support with specific data/detail
- Third sentence: Add broader context/impact
- Maximum 3 short paragraphs
- Each paragraph = one distinct point
- No empty phrases like:
  • "as events unfold"
  • "amid escalating"
  • "recent developments"
  • "prediction markets indicate"
- No qualifiers like:
  • "staggering"
  • "dramatic"
  • "significant"
  • "unprecedented"
- Lead with data/numbers when they're the news
- Cut all prepositional phrases that don't add value
- Use active voice
- 12-15 words per sentence maximum

FORMAT:
[HEADLINE] (8 words max)
[SUBHEADER] (1 line)
[BLURB] (3 paragraphs max)

Ensure that you use the exact formatting with the bold markers (**) as shown above.

Now, using the provided market_dict and articles, generate a compelling newsletter section following the format above. Remember to focus on the most intriguing aspects of the prediction market, potentially contrasting with news reports if relevant. Craft a headline that captures attention and sparks curiosity, always ensuring your content accurately reflects the time frame specified in the market. If the provided web searches aren't relevant, feel free to focus solely on the fascinating world of the prediction market itself. Keep in mind, that you are going to be asked to do many of these articles in a row, so make your wording unique so you do not repeat phrasings.

Input Market Info:
{market_dict}

Important market analysis to utilize:
{response_dict}

Input Context from Web:
{context}
"""
    better_llm = ChatOpenAI(model="gpt-5", api_key=openai_api, temperature=1)
    structured_llm_better = better_llm.with_structured_output(ArticleOutput)
    response = structured_llm_better.invoke([HumanMessage(content=prompt)])
    print("this is the stuctured article output: ", response.dict())

    system_prompt = f"""You are an expert editor evaluating newsletter content that combines traditional news with prediction market insights from Polymarket. You'll score submissions from 0-10 based on three key criteria:

1. Insight Value (0-10):
   - Does it reveal non-obvious implications using prediction market data?
   - Does it connect market probabilities to real-world implications?
   - Does it go beyond surface-level news reporting?
   - Is it on an interesting topic? (Sports scores are not interesting. They should get low scores except for occurances where it is beyond predicting a matchup's outcome). Juicy pop culture surprises can be pretty interesting.

2. Information Quality (0-10):
   - Is the information specific and actionable?
   - Is it revealing information that otherwise wouldnt be clear? If it is on a topic in the news with uncertainty, the market could be a big help.
   - Are market probabilities clearly connected to conclusions?
   - Does it avoid vague or generic statements?
   - If it is predicting something at near 100 percent certainty, then it is not that interesting.

3. Professionalism & Tone (0-10):
   - Is the content respectful and professional?
   - Does it maintain appropriate formality when discussing serious topics?
   - Does it avoid sensationalism while remaining engaging?

The final score is the average of these three scores, rounded to the nearest integer.

Here are examples of different quality levels:

SCORE 10 EXAMPLE:
Headline: "Market Signals 75% Chance of Fed Rate Pause Through Q2"
Subheader: "Traders Betting Heavily on Extended Hold Pattern Despite Recent Inflation Data"
Blurb: "While mainstream coverage focuses on the latest CPI print, Polymarket's fed-futures contract reveals traders are increasingly confident about an extended rate pause. The 75% probability suggests markets have largely priced in stabilizing inflation, despite mixed economic signals. This diverges from traditional analyst forecasts, which remain split on the Fed's next move. For investors, this implies potential opportunities in rate-sensitive sectors that may be mispriced based on mainstream narratives."

Analysis:
- Insight Value: 10/10 - Combines market data with practical investment implications
- Information Quality: 10/10 - Specific probabilities and actionable insights
- Professionalism: 10/10 - Maintains formal tone while being engaging
Total Score: 10/10

SCORE 5 EXAMPLE:
Headline: "Ukraine Aid Package Shows 60% Chance of Passage"
Subheader: "Market Odds Reflect Congressional Uncertainty"
Blurb: "Polymarket traders are giving the latest Ukraine aid package a 60% chance of passing this month. This matches the general consensus from political analysts, though doesn't provide much additional insight. The market has fluctuated between 55-65% over the past week, suggesting no clear direction yet."

Analysis:
- Insight Value: 4/10 - Mostly restates known information
- Information Quality: 6/10 - Clear probabilities but limited analysis
- Professionalism: 5/10 - Adequate but could provide more context
Total Score: 5/10

SCORE 0 EXAMPLE:
Headline: "Markets Predict Chaos in Upcoming Election!!!"
Subheader: "Traders Betting Big on Political Drama"
Blurb: "Election markets are going crazy right now! Some traders are betting everything will fall apart, while others think it'll be fine. Numbers are all over the place. Could be really bad news for everyone if things go wrong!!!"

Analysis:
- Insight Value: 0/10 - No specific insights or analysis
- Information Quality: 0/10 - Vague statements without data
- Professionalism: 0/10 - Sensationalist and unprofessional tone
Total Score: 0/10

Please evaluate the following content and provide a single integer score from 0-10, reflecting the average of the three criteria rounded to the nearest integer. Format your response as JSON with a single key 'score' and an integer value.

Content to evaluate: 
Headline: {response.headline}
Subheader: {response.subheader}
Blurb: {response.blurb}
"""
    score_llm = llm.with_structured_output(article_score)
    score_response = score_llm.invoke([HumanMessage(content=system_prompt)])

    print("returning article with headline: ", response.headline)
    if score_response.score > 6:
      return {"newsletter_content" : [ArticleOutput(
              headline=response.headline,
              subheader=response.subheader,
              blurb=response.blurb,
              score=score_response.score,
              ticker = market.ticker,
              links = market.links
          )]}
    else:
       return {"skipped_newsletter_content" : [ArticleOutput(
              headline=response.headline,
              subheader=response.subheader,
              blurb=response.blurb,
              score=score_response.score,
              ticker = market.ticker,
              links = market.links
          )]}

In [13]:
def continue_to_markets(state: OverallGraph):
    return [Send("parallel_article_writing", market) for market in state.markets_list]

In [14]:
class ArticleGroups(BaseModel):
    article_groups: Dict[str, List[int]] = Field(
        default_factory=dict,
        description="Dictionary of lists, where each list contains ids of a group of articles. The string is the title of the group"
    )

In [ ]:
def enumerate_articles(articles):
    return "\n".join(f"{article}" for article in articles)

def generate_groups(state: OverallGraph):
    llm = ChatOpenAI(model="gpt-5", api_key=openai_api, temperature=1)  # Fixed model name
    structured_llm = llm.with_structured_output(ArticleGroups)
    
    formatted_articles = "\n".join([
        f"ID {article.id}: {article.headline}" 
        for article in state.newsletter_content
    ])
    
    prompt = f"""You are organizing articles for a prediction market-focused newsletter. 
Group these articles into standard news categories that would appear in a major newspaper.

Guidelines for categories:
1. Use quippy, interesting section names that group as many articles as possible
2. Requirements:
   - Each article must be in exactly one category
   - Each category must contain 2-6 related articles. Always at least 3.
   - Categories should be broad enough to group related stories
   - Put every article in a category.

Example format:
{{
    "article_groups": {{
        "US Politics": [1, 4, 7],
        "Elon's Companies": [2, 5, 8],
        "NFL Playoffs": [3, 6, 9]
    }}
}}

Articles to categorize:
{formatted_articles}

Create appropriate groupings where each article ID is used exactly once."""
    
    try:
        print(prompt)
        result = structured_llm.invoke([SystemMessage(content=prompt)])
        print("Structured output:", result)
        return {"article_groups": result.article_groups}
    except Exception as e:
        print(f"Error generating groups: {e}")
        return {"article_groups": {}}
def send_redundant_groups(state : OverallGraph):
    if state.done == True:
        print("ending")
        print("state before ending: ", state)
        return END
    list_of_lists_of_articles =[]
    for redundant_group in state.redundant_groups:
        articles_idx_individual = [state.newsletter_content[index] for index in redundant_group]
        list_of_lists_of_articles.append(articles_idx_individual)
    print(list_of_lists_of_articles)
    return [Send("combine_articles", articles) for articles in list_of_lists_of_articles]
def combine_articles(articles):
    print('fixing this redundant group')
    llm = ChatOpenAI(model="gpt-5", api_key=openai_api, temperature=1)
    structured_llm = llm.with_structured_output(ArticleOutput)
    prompt = f"""
    You are a skilled AI editor tasked with condensing multiple prediction market newsletter articles into a single, comprehensive article. Your goal is to distill the most crucial information from each input article while maintaining the engaging style and format of the original pieces.
    Input:
    You will receive a list of newsletter articles, each following this structure:
    Headline: [Attention-grabbing headline]
    Subheader: [Supporting subheader adding context]
    Blurb: [2-3 sentence paragraph expanding on the headline and subheader]
    Instructions:

    Analyze all input articles, identifying the most significant and intriguing information from each.
    Create a new, overarching headline that captures the essence of the combined articles.
    Craft a subheader that provides additional context for the main headline.
    Write a concise blurb that synthesizes the key points from all input articles. This blurb should:

    Highlight the most interesting prediction market probabilities and movements.
    Mention any relevant time frames or end dates for the markets.
    Include any notable contrasts with traditional news reporting, if applicable.
    Preserve the confident, slightly irreverent tone of the original articles.


    Ensure your writing is engaging and assumes an audience familiar with prediction markets.
    Be as concise as possible while encoding the maximum amount of relevant information.

    Your output should follow this format:
    Headline: [Your new, overarching headline]
    Subheader: [A supporting subheader that adds context]
    Blurb: [A 3-4 sentence paragraph synthesizing key information from all input articles]
    Remember to use the exact formatting with the bold markers (**) as shown above.
    Now, using the provided input articles, generate a single, comprehensive newsletter article that captures the most crucial information while maintaining the engaging style and format of the original pieces.
    Here are the articles:
    {articles}"""
    response = structured_llm.invoke([SystemMessage(content=prompt)])
    print("made this new article out of ", len(articles), " articles with the headline: ", response.headline)
     
    return {"newsletter_content" : [ArticleOutput(
            headline=response.headline,
            subheader=response.subheader,
            blurb=response.blurb
        )]}
def remove_redundants(state: OverallGraph):
    print("removing redundants")
    flat_indices = set([index for sublist in state.redundant_groups for index in sublist])
    # Remove items from each sublist
    state.newsletter_content = [item for i, item in enumerate(state.newsletter_content) if i not in flat_indices]
    return state

In [16]:
def check_if_done(state : OverallGraph):
    if state.done == True:
        return END
    else:
        return 'combine_similar'
    
def deduplicate_articles(state: OverallGraph):
    return [Send("parallel_article_writing", market) for market in state.markets_list]

In [17]:
# Define the graph
from IPython.display import Image, display
def define_workflow():
    workflow = StateGraph(MarketInfo)
    
    workflow.add_node("get_context", tavily_search)
    workflow.add_node("generate_newsletter", generate_newsletter_content)
    
    workflow.set_entry_point("get_context")
    workflow.add_edge("get_context", "generate_newsletter")
    workflow.add_edge("generate_newsletter", END)

    parallel_workflow = StateGraph(OverallGraph)

    parallel_workflow.add_node("parallel_article_writing", workflow.compile())
    parallel_workflow.set_conditional_entry_point(continue_to_markets, ["parallel_article_writing"])
    parallel_workflow.add_node("generate_groups", generate_groups)
    parallel_workflow.add_edge("parallel_article_writing", "generate_groups")
    # parallel_workflow.add_node("combine_articles", combine_articles)
    # parallel_workflow.add_conditional_edges("generate_groups", send_redundant_groups, ["combine_articles", END])
    # parallel_workflow.add_node("remove_redundants", remove_redundants)
    # parallel_workflow.add_edge("combine_articles", "remove_redundants")
    parallel_workflow.add_edge("generate_groups", END)
    react_graph = parallel_workflow.compile()
    


    

    # Show
    #display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))
    
    return react_graph

In [30]:
def articles_wrapper(days_in_past):
    market_info_list = []
    graph = define_workflow()
    articles = []
    full_info = []
    markets = get_markets_for_date_range(days_in_past=days_in_past, limit=75)
    excluded_tags = ["Crypto", "Weather", "Mentions", "Sports"]
    filtered_markets = [market for market in markets if not any(tag['label'] in excluded_tags for tag in market.get('tags', []))]
    sorted_markets_by_interest = sorted(
    {market['title']: market for market in filtered_markets if market.get('interest_score', 0) > 100}.values(),
    key=lambda x: x.get('interest_score', 0),
    reverse=True
)
    print("len of markets post score cutoff: ", len(sorted_markets_by_interest))
    for market in sorted_markets_by_interest:
        market_info_example = MarketInfo(**market)
        # market_end_result = app.invoke(market_info_example)
        market_info_list.append(market_info_example)
        # if market_end_result['newsletter_content']:
        #     articles.append(market_end_result['newsletter_content'])
        #     full_info.append(market_end_result)
    print("number of markets going into the graph: ",len(market_info_list))
    markets_model = OverallGraph(markets_list=market_info_list)
    result = graph.invoke(markets_model)
    
    return result

In [31]:
result = articles_wrapper(21)


len of markets post score cutoff:  17
number of markets going into the graph:  17
['https://en.wikipedia.org/wiki/Opinion_polling_for_the_2025_Dutch_general_election', 'https://nos.nl/collectie/14002/artikel/2586741-peilingwijzer-groei-d66-zet-door-veel-kiezers-zweven-nog', 'https://nos.nl/collectie/14002/artikel/2585755-peilingwijzer-pvv-duidelijk-de-grootste-lichte-winst-d66-en-ja21', 'https://eenvandaag.avrotros.nl/opiniepanel/uitslagen/groenlinks-pvda-door-verlies-in-laatste-zetelpeilingen-op-grotere-afstand-van-pvv-veel-kiezers-zweven-nog-161486', 'https://en.wikipedia.org/wiki/2025_Dutch_general_election', 'https://eenvandaag.avrotros.nl/opiniepanel/uitslagen/pvv-verliest-licht-in-nieuwe-zetelpeiling-d66-ja21-en-vvd-nu-op-gelijke-hoogte-161636', 'https://www.euronews.com/my-europe/2025/10/28/dutch-elections-five-things-to-watch', 'https://www.atlanticcouncil.org/content-series/eye-on-europes-elections/your-primer-on-the-dutch-general-elections/', 'https://www.politico.eu/europe-p

/Users/ethangoldberg/Desktop/CS-Proj/PolyNewsDaily/.venv/lib/python3.13/site-packages/langgraph/utils/runnable.py:405: UserWarning: WARNING! parallel_tool_calls is not default parameter.
                parallel_tool_calls was transferred to model_kwargs.
                Please confirm that parallel_tool_calls is what you intended.
  input = context.run(step.invoke, input, config, **kwargs)


['https://polymarket.com/event/elon-musk-of-tweets-october-28-november-4', 'https://polymarket.com/event/elon-musk-of-tweets-october-28-november-4/elon-musk-of-tweets-october-28-november-4-400-419', 'https://polymarket.com/event/elon-musk-of-tweets-october-21-october-28', 'https://polymarket.com/event/elon-musk-tweets-25-times-wednesday', 'https://x.com/i/grok/share/BKqOPnS57asr07mAveDKArVkd', 'https://myriad.markets/markets/will-elon-musk-publish-over-220-x-posts-between-may-30-and-june-6-d1e4297b-f842-4ae0-8784-d845113ed0c3', 'https://www.xtracker.io/', 'https://www.kaggle.com/datasets/dadalyndell/elon-musk-tweets-2010-to-2025-march', 'https://www.semafor.com/article/05/14/2025/elon-musk-posted-half-as-many-xeets-in-april', 'https://www.cnbc.com/2025/10/17/proxy-advisor-iss-opposes-tesla-ceo-elon-musk-1-trillion-pay-plan.html']
['<Document href="https://polymarket.com/event/elon-musk-of-tweets-october-28-november-4"/>\nCommunity reposts which are not counted by the tracker not count 

/Users/ethangoldberg/Desktop/CS-Proj/PolyNewsDaily/.venv/lib/python3.13/site-packages/langgraph/utils/runnable.py:405: UserWarning: WARNING! parallel_tool_calls is not default parameter.
                parallel_tool_calls was transferred to model_kwargs.
                Please confirm that parallel_tool_calls is what you intended.
  input = context.run(step.invoke, input, config, **kwargs)
/Users/ethangoldberg/Desktop/CS-Proj/PolyNewsDaily/.venv/lib/python3.13/site-packages/langgraph/utils/runnable.py:405: UserWarning: WARNING! parallel_tool_calls is not default parameter.
                parallel_tool_calls was transferred to model_kwargs.
                Please confirm that parallel_tool_calls is what you intended.
  input = context.run(step.invoke, input, config, **kwargs)


['https://trends.withgoogle.com/year-in-search/data-methodology/', 'https://trends.withgoogle.com/en-gb/year-in-search/2024/us/', 'https://trends.withgoogle.com/year-in-search/2024/in/', 'https://trends.withgoogle.com/year-in-search/2024/ng/', 'https://trends.withgoogle.com/year-in-search/2024/ro/', 'https://trends.withgoogle.com/year-in-search/2024/ca/', 'https://trends.withgoogle.com/year-in-search/2024/us/', 'https://trends.withgoogle.com/year-in-search/2023/', 'https://trends.withgoogle.com/year-in-search/2024/', 'https://trends.withgoogle.com/en-au/year-in-search/2024/us/']
['<Document href="https://trends.withgoogle.com/year-in-search/data-methodology/"/>\nGoogle Trends provides a unique look at what the world is searching for, and what it seeks to better understand. Year in Search reveals the breakout questions, moments, and individuals the world searched for throughout the year. These searches reflect specific moments in time and illuminate our shared experiences and cultural e

...

this is the intermediate reseponse:  {'what_is_the_market_asking_or_answering': 'Which athlete will be ranked #1 in Google’s official Year in Search list for 2025, as published by Google’s Year in Search hub (specifically, the Global → Sports → Athletes ranking)?', 'important_criteria': ['The market resolves according to the athlete ranked #1 on Google’s official Year in Search list for 2025.', 'Resolution source: trends.withgoogle.com/year-in-search (Global → Sports → Athletes).', 'Resolution will occur as soon as Google releases its official 2025 Year in Search data, but no later than March 31, 2026, 11:59 PM ET.', "If none of the listed options is #1 (or if no Google data is released by the deadline), the market resolves to 'Other'.", 'Nicknames/alternate spellings for athletes are accepted as referring to the intended person.'], 'completed': False, 'important_insights_context_or_reasoning': "Recent Google Year in Search data (for 2023 and 2024) provide historical context for how un

/Users/ethangoldberg/Desktop/CS-Proj/PolyNewsDaily/.venv/lib/python3.13/site-packages/langgraph/utils/runnable.py:405: UserWarning: WARNING! parallel_tool_calls is not default parameter.
                parallel_tool_calls was transferred to model_kwargs.
                Please confirm that parallel_tool_calls is what you intended.
  input = context.run(step.invoke, input, config, **kwargs)


this is the intermediate reseponse:  {'what_is_the_market_asking_or_answering': "This market is asking which publicly-listed company will be the world's second-largest by market capitalization as of the close of trading on December 31, 2025. The resolution will rely on a consensus from credible reporting sources.", 'important_criteria': ['Resolution date is December 31, 2025, based on the company market capitalizations at official market close.', 'Market cap rankings must be based on consensus credible financial reporting (e.g., Forbes, Bloomberg, Reuters, PwC, etc.).', 'Only public companies are considered.', 'Market is binary per option (only the company ranked exactly 2nd by market cap is the winner; no ties are specified).', 'Current market probabilities are: Microsoft 50.5%, Apple 42.5%, Alphabet 6.25%, NVIDIA 2.85%, Tesla 1.8%, Saudi Aramco 0.3%, Amazon 0.3%, Oracle 0.3%, all others near zero or zero, indicating market belief that only a handful of major technology companies are 

/Users/ethangoldberg/Desktop/CS-Proj/PolyNewsDaily/.venv/lib/python3.13/site-packages/langgraph/utils/runnable.py:405: UserWarning: WARNING! parallel_tool_calls is not default parameter.
                parallel_tool_calls was transferred to model_kwargs.
                Please confirm that parallel_tool_calls is what you intended.
  input = context.run(step.invoke, input, config, **kwargs)


this is the intermediate reseponse:  {'what_is_the_market_asking_or_answering': "This market asks which album will win 'Best Rap Album' at the 68th Annual GRAMMY Awards, scheduled for February 1, 2026.", 'important_criteria': ["The market will resolve to the album that wins 'Best Rap Album' at the 68th GRAMMY Awards (Feb. 1, 2026).", 'If no winner is declared by June 30, 2026, or if there is a tie, the market will resolve in favor of the listed album which comes first in alphabetical order.', 'Resolution source: The televised GRAMMY Awards ceremony, the official GRAMMY website, and other reputable reporting sources.', 'The eligibility period for the award is August 31, 2024 to August 30, 2025.', 'Nominees are set to be announced November 7, 2025, with final voting from December 12, 2025 to January 5, 2026.'], 'completed': False, 'important_insights_context_or_reasoning': "Current market probabilities indicate Kendrick Lamar's 'GNX' (44%) and Clipse's 'Let God Sort Em Out' (36%) as fron

In [19]:
newsletter_content =  [ArticleOutput(id=23, headline='Iran Military Action Against Israel Likely in 2024', subheader='Prediction market signals 58% chance of Iran-initiated conflict', blurb="The prediction market now assigns a 58% probability to Iran launching a military action against Israel by the end of 2024. This slight uptick in probability highlights the ever-present tensions between the two nations and reflects traders' concerns about potential escalation. With active trading volumes, the market underscores the significance of geopolitical developments in the Middle East.", score=5, ticker='another-iran-strike-on-israel-in-2024', links=['nothing.com']),
  ArticleOutput(id=29, headline='Kamala Harris Faces 8.45% Odds for Exact 270 Electoral Votes', subheader="Prediction market highlights challenging path through 'blue wall' states", blurb="As the 2024 U.S. Presidential Election approaches, prediction markets give Kamala Harris an 8.45% chance of securing exactly 270 electoral votes by winning a specified set of states, including the pivotal 'blue wall' states of Michigan, Pennsylvania, and Wisconsin. Despite a slight increase in confidence, market skepticism remains high. The outcome depends on her ability to navigate shifting political dynamics and voter turnout in these key regions.", score=6, ticker='kamala-wins-with-blue-wall-mi-pa-wi-270-evs', links=['nothing.com']),
  ArticleOutput(id=13, headline='Israel Withdrawal from Lebanon Faces Uncertainty', subheader="Market shows 42% chance of Israel's withdrawal by November 30", blurb='The prediction market suggests a 42% likelihood that Israel will announce a full withdrawal from Lebanon before the end of November 2024. This probability has decreased by 12.5% recently, reflecting growing skepticism amid ongoing regional tensions. Despite the challenges, the outcome remains closely watched as geopolitical dynamics continue to evolve.', score=5, ticker='israel-withdraws-from-lebanon-before-december', links=['nothing.com']),
  ArticleOutput(id=28, headline='Ethereum Unlikely to Hit $2,600 by November 1', subheader='Market shows low confidence in Ethereum price surge', blurb="With a mere 10.5% probability, the prediction market indicates skepticism about Ethereum reaching $2,600 by noon ET on November 1, 2024. The slight 0.6% drop in confidence over the last day further emphasizes a cautious sentiment among traders. As the resolution date approaches, investors should stay vigilant of any market shifts that could impact Ethereum's trajectory.", score=6, ticker='ethereum-above-2600-on-november-1', links=['nothing.com']),
  ArticleOutput(id=26, headline='Solana Faces Uncertainty for $170 Mark by November 2024', subheader="Crypto Traders Show Mixed Sentiment on Solana's Future", blurb="As of now, Solana's chances of trading above $170 on November 1, 2024, stand at a moderate 43.5%. The slight decrease in probability over the past day highlights growing uncertainty in the market, possibly due to recent crypto trends or news affecting investor confidence. With the volatile nature of cryptocurrencies like Solana, the final outcome remains unpredictable, making this market a captivating watch for crypto enthusiasts.", score=5, ticker='solana-above-170-on-november-1', links=['nothing.com']),
  ArticleOutput(id=32, headline='Trump Albuquerque Rally Highlights China, Biden Amid Election Campaign', subheader="Prediction market expects Trump to focus on China, Biden, and 'Garbage' in rally", blurb="As Donald Trump prepares for his Albuquerque rally, prediction markets are buzzing with expectations. Traders overwhelmingly predict he will spotlight issues like China and Biden, with probabilities soaring to 99.95%. This aligns with his ongoing narrative as he gears up for the 2024 presidential election campaign. Meanwhile, topics such as 'Native American' and 'Puerto Rico' are seen as unlikely to feature prominently, reflecting a strategic focus in his messaging.", score=7, ticker='what-will-trump-say-during-albuquerque-nm-rally', links=['nothing.com']),
  ArticleOutput(id=14, headline='Bitcoin eyes record high before US election', subheader='Market predicts 30% chance amidst price decline', blurb='As the US presidential election approaches, Bitcoin faces a potential new all-time high, with markets assigning a 30% probability of surpassing $73,777 by November 4, 2024. This prediction comes after a notable 33.5% drop in probability, suggesting shifting sentiments. With cryptocurrency dynamics intertwined with political events, investors should brace for potential volatility.', score=6, ticker='bitcoin-new-all-time-high-before-election', links=['nothing.com']),
  ArticleOutput(id=27, headline='Bitcoin poised to surpass $68000 by November 1', subheader='Market predicts strong bullish sentiment despite slight dip', blurb="The Polymarket prediction indicates a 92% likelihood that Bitcoin will exceed $68,000 by November 1, 2024, according to Binance's trading data. This optimistic forecast reflects a robust bullish sentiment in the crypto market, even though the probability dipped slightly by 1.85% recently. Investors are banking on historical trends and favorable market conditions to push Bitcoin past this significant price threshold.", score=8, ticker='bitcoin-above-68000-on-november-1', links=['nothing.com']),
  ArticleOutput(id=22, headline='Bitcoin unlikely to hit $80k before election', subheader='Market skepticism grows as probability drops to 6%', blurb='The Polymarket prediction market indicates a mere 6% chance of Bitcoin reaching $80,000 before the U.S. election. This skepticism is reflected in a recent drop in probability, suggesting traders doubt significant price surges amidst current market conditions. Factors such as market sentiment and potential regulatory changes could be influencing this outlook.', score=6, ticker='bitcoin-hits-80k-before-election', links=['nothing.com']),
  ArticleOutput(id=30, headline="Trump Rally Likely Features 'Women's Sports' and 'Pocahontas'", subheader='Prediction markets signal key phrases for Nevada event', blurb="As former President Donald Trump prepares for his rally in Henderson, Nevada, prediction markets are buzzing with expectations of what he might say. Top contenders include 'Women's Sports' and 'Pocahontas,' each with a high probability of being mentioned. These phrases align with Trump's previous campaign themes, suggesting they will resonate with his audience as he gears up for the 2024 election. Meanwhile, terms like 'Crypto/Bitcoin' and 'Tampon' are far less likely, reflecting a strategic focus on traditional issues.", score=5, ticker='what-will-trump-say-during-nevada-rally-oct-31', links=['nothing.com']),
  ArticleOutput(id=2, headline='Trump Faces Steep Challenge in Florida', subheader='Prediction Market Shows Declining Confidence in 12-point Margin', blurb="As the 2024 U.S. Presidential Election approaches, the Polymarket prediction market indicates a mere 20.5% chance that Donald Trump will secure Florida by a 12-point margin, a key battleground state. This reflects a notable decrease in confidence, with probabilities dropping 0.11 in just one day. With Florida's pivotal role in the election, this shift highlights the uncertainty and fierce competition expected in the upcoming race.", score=6, ticker='will-trump-win-florida-by-12-points', links=['nothing.com']),
  ArticleOutput(id=20, headline='Texans Face Tight Odds Against Jets', subheader='Market Predicts Competitive Matchup in Upcoming NFL Game', blurb='As the Houston Texans prepare to face the New York Jets on October 31, prediction markets reflect a closely contested game with the Texans holding a 45.5% chance of victory. Recent trading activity shows a slight dip in confidence for the Texans, dropping 2% in probability. With the game still days away, factors like player health and team dynamics could sway the odds further.', score=5, ticker='nfl-hou-nyj-2024-10-31', links=['nothing.com']),
  ArticleOutput(id=19, headline='Harris Expected to Win New Jersey by 10% Margin', subheader="Prediction market shows strong confidence in Harris's lead", blurb='With over a year until the 2024 U.S. Presidential Election, prediction markets show a 79.5% probability that Kamala Harris will secure a 10% or greater margin of victory in New Jersey. This reflects strong confidence in her candidacy despite the potential for political developments to influence public opinion before the election date. The market will remain open as it awaits the official vote count and certification in New Jersey.', score=8, ticker='will-harris-win-new-jersey-by-10-points', links=['nothing.com']),
  ArticleOutput(id=4, headline='Trump faces uphill battle in Rust Belt sweep', subheader='Prediction market shows 26% chance for Trump victory in key states', blurb='As the 2024 US Presidential Election approaches, prediction markets indicate a challenging path for Donald Trump in the Rust Belt swing states of Wisconsin, Michigan, and Pennsylvania. With only a 26% probability of sweeping these crucial states, the market reflects skepticism about his ability to replicate past successes. This development is critical, as these states often play a pivotal role in election outcomes. Stay tuned as the dynamics evolve closer to election day.', score=6, ticker='trump-rust-belt-swing-state-sweep', links=['nothing.com']),
  ArticleOutput(id=18, headline='Trump Faces Uphill Battle in Iowa Poll', subheader="Prediction Market Shows Low Confidence in Trump's Dominance", blurb="As the Iowa 2024 presidential election approaches, the prediction market shows only a 13.5% probability that Donald Trump will lead by a 12% margin or more in Anne Selzer's final poll. Despite a slight 0.5% increase in confidence, market participants remain skeptical of Trump's ability to secure a commanding lead. This reflects a cautious stance among observers, potentially due to recent campaign dynamics or shifts in voter opinion.", score=6, ticker='trump-12-in-selzer-iowa-poll', links=['nothing.com']),
  ArticleOutput(id=25, headline='Chase Oliver Leads 2024 Third-Party Race', subheader='Prediction market shows Oliver ahead of Stein and RFK Jr.', blurb='In the latest insights from Polymarket, Chase Oliver emerges as the frontrunner among third-party candidates for the 2024 US Presidential election, holding a 38% probability of winning the most votes. Jill Stein follows closely with 33%, while RFK Jr. has a 27% chance. Interestingly, all candidates have experienced a dip in their probabilities, indicating recent shifts in voter sentiment or polling data. Despite their presence, Cornel West, Vermin Supreme, and other candidates are unlikely to make a significant impact, as reflected in their low probabilities.', score=7, ticker='which-3rd-party-candidate-wins-most-votes', links=['nothing.com'])],
article_groups = {'Middle East Politics': [23, 13],
  'US Election Predictions': [29, 32, 2, 19, 4, 18, 25],
  'Cryptocurrency Forecasts': [28, 26, 14, 27, 22],
  'Trump Rallies': [30, 4, 18],
  'Sports and Betting': [20]}

In [33]:
articles = result["newsletter_content"]
# "vikmanocha@gmail.com", "jameswmarren@gmail.com"
email_list = ["ethanagoldberg@gmail.com"]
groups = result["article_groups"]
result

{'markets_list': [MarketInfo(interest_score=91906.31371771479, title='Israel x Hamas ceasefire cancelled by...?', ticker='israel-x-hamas-ceasefire-cancelled-by-october-31', description='On October 9, Israel and Hamas signed a deal implementing a ceasefire: https://www.reuters.com/world/middle-east/israel-hamas-agree-gaza-ceasefire-return-hostages-2025-10-09/\n\nThis market will resolve to "Yes" if a either Israel or Hamas announce the cancellation of the ongoing ceasefire, or if a consensus of credible reporting confirms that the ceasefire is no longer in effect, by October 31, 2025, 11:59 PM ET. Otherwise this market will resolve to "No".\n\nAnnouncements that one side or another violated the ongoing ceasefire will not be enough to resolve this market to "Yes"; only a definitive announcement by Israel or Hamas, or otherwise a wide consensus of credible reporting that that the ceasefire has been ended, will qualify to resolve this market to "Yes".\n\nThe primary solution resolution sou

In [21]:
from newsletter_sender import NewsletterSender, test_smtp_connection
from typing import List
smtp_config = {
    "host": "smtp.gmail.com",
    "port": 587,
    "secure": True,
    "auth": {
        "user": "polynewsdailynewsletter@gmail.com",  # Your full Gmail address
        "pass": smtp_pass    # The 16-character app password you generated
    },
    "from": '"PolyNewsDaily Update" <polynewsdailynewsletter@gmail.com>'  # Use your Gmail address here too
}

In [22]:
print("articles:", articles)
print("groups:", groups)

articles: [ArticleOutput(id=4, headline='Israel Commits 84% Probability to Gaza Ground Offensive', subheader="Market surge reflects Israeli cabinet's expanded offensive plans and ongoing ground attacks", blurb='Prediction market signals 84% chance Israel launches major ground offensive in Gaza this May. Probability jumped 23.5 points in 24 hours, driven by Israeli cabinet approval for offensive expansion and intensified ground assaults in northern Gaza. These military moves aim to seize the entire Gaza Strip, significantly impacting regional stability and conflict dynamics through May 31, 2025.', score=7, ticker='will-israel-launch-a-major-ground-offensive-in-gaza-in-may', links=['https://en.wikipedia.org/wiki/March_2025_Israeli_attacks_on_the_Gaza_Strip', 'https://www.aljazeera.com/news/liveblog/2025/5/14/live-israel-attacks-gaza-hospitals-as-trump-says-working-to-end-war-soon', 'https://www.jns.org/a-year-plus-of-combat-in-gaza-is-needed/', 'https://www.bbc.com/news/articles/crr704ww

In [23]:
if test_smtp_connection(smtp_config):
    print("Connection successful! Sending test newsletter...")
    
    # Create sender and send newsletter
    sender = NewsletterSender()
    email_list = sender.get_subscriber_emails()
    if email_list:
        save_result = sender.save_newsletter_to_db(articles, groups)
        results = sender.send_newsletter(smtp_config, email_list, articles, groups)
    else:
        email_list = ["ethanagoldberg@gmail.com"]
        save_result = sender.save_newsletter_to_db(articles, groups)
        results = sender.send_newsletter(smtp_config, email_list, articles, groups)

✅ SMTP connection successful!
Connection successful! Sending test newsletter...
Connecting to: postgres://u8tpitnkehg4bm@c1i13pt05ja4ag.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com:5432/d8714o8m60rj7k


Found 45 subscribers
Connecting to: postgres://u8tpitnkehg4bm@c1i13pt05ja4ag.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com:5432/d8714o8m60rj7k



    Database summary:
    - Articles: 10
    - Groups: 4
    - Group-Article associations: 13
            
Newsletter saved to database successfully


Sending batch 1/1


Successfully sent to: lgoldbergcourt@gmail.com


...

Successfully sent to: dylanhurwitz@gmail.com


Successfully sent to: maniya.chowdhary21@gmail.com


Successfully sent to: billweica@yahoo.com


Successfully sent to: mli0118999@gmail.com


Successfully sent to: maxwellyoas@gmail.com
